
# 🔧 NLP Autocomplete — LSTM & DistilGPT2 (ROUGE)

Этот ноутбук показывает запуск:
1. Обучение LSTM (`lstm_train.py`) на задаче предсказания последнего токена.
2. Оценку DistilGPT2 (`eval_transformer_pipeline.py`) для дописывания **последней 1/4** текста с подсчётом **ROUGE**.

> Файлы берутся из текущего проекта (`/mnt/data`). Предполагается, что зависимости `torch`, `transformers`, `evaluate`, `pandas`, `tqdm` уже установлены.


# 0) Перевод .txt в csv (train/val/test)

In [1]:
from src import split_dataset
split_dataset.main()


OK: 1600498 строк сохранено в data/raw_data.csv
Датасет очищен
Длина train - 1280398 
Длина val - 160050 
Длина test - 160050
Датасет успешно разделен и очищен


## 1) Проверка окружения

In [1]:
import os, sys

print("CWD:", os.getcwd())
try:
    import torch, transformers, evaluate, pandas
    print("PyTorch:", torch.__version__)
    print("Transformers:", transformers.__version__)
    print("evaluate:", evaluate.__version__)
    print("pandas:", pandas.__version__)
except Exception as e:
    print("Проверьте зависимости:", e)

# смотрим локальную папку src рядом с ноутбуком
proj_src = os.path.join(os.getcwd(), "src")
print("\nПуть к src:", proj_src, "| exists:", os.path.isdir(proj_src))
if proj_src not in sys.path:
    sys.path.insert(0, proj_src)  # добавляем src в sys.path

print("\nФайлы в src:")
print(os.listdir(proj_src))


CWD: /root/code


/root/code/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch: 2.9.0+cu128
Transformers: 4.57.1
evaluate: 0.4.6
pandas: 2.3.3

Путь к src: /root/code/src | exists: True

Файлы в src:
['lstm_train.py', 'eval_lstm.py', '__pycache__', 'lstm_model.py', 'data_utils.py', 'eval_transformer_pipeline.py', 'next_token_dataset.py', 'split_dataset.py']


## 2) Проверка данных

In [2]:

import os, pandas as pd
base = "data/"
os.makedirs(base, exist_ok=True)
for fname in ["train.csv","val.csv","test.csv"]:
    path = os.path.join(base, fname)
    print(fname, "exists:", os.path.exists(path))
    if os.path.exists(path):
        df = pd.read_csv(path)
        print("  rows:", len(df), "| cols:", list(df.columns))
        print(df.head(3))


train.csv exists: True
  rows: 1280398 | cols: ['text']
                                                text
0  @skinnyribs  why must you sell that itokin pie...
1  @mizsprieta i have &amp; it still wont go away...
2                 I'm swag surfing on my way to work
val.csv exists: True
  rows: 160050 | cols: ['text']
                                                text
0  @CaityPineapple Yumm!  Argh I just left home t...
1            anybody want 2 chat w/ me?? im bored...
2          i could go 4 a back rub.....    somebody?
test.csv exists: True
  rows: 160050 | cols: ['text']
                                                text
0  nightt worldd...soon 2 be in the once place i ...
1  off work, went jetskiing, doing this then read...
2  @Gen215 ROFL following Jesus! Found 1 th othr ...


## 3) Обучение LSTM (`lstm_train.py`)

In [3]:
# Импорт и запуск
import os, sys, importlib
import evaluate, numpy as np

proj_src = os.path.join(os.getcwd(), "src")
if proj_src not in sys.path:
    sys.path.append(proj_src)

import lstm_train  
importlib.reload(lstm_train)

print("Запуск lstm_train.main() ...")
lstm_train.main()


Запуск lstm_train.main() ...
LSTM parameters count -  12146234


100%|██████████| 19983/19983 [03:06<00:00, 107.42it/s]


Epoch 1 | Train Loss: 5.296 | Val Loss: 4.935 | Val Accuracy: 25.67%
 Сохранен чекпоинт в: models/lstm_best.pt (val_loss=4.9354)


100%|██████████| 19983/19983 [03:06<00:00, 107.36it/s]


Epoch 2 | Train Loss: 4.648 | Val Loss: 4.836 | Val Accuracy: 26.81%
 Сохранен чекпоинт в: models/lstm_best.pt (val_loss=4.8363)


100%|██████████| 19983/19983 [03:05<00:00, 107.46it/s]


Epoch 3 | Train Loss: 4.448 | Val Loss: 4.828 | Val Accuracy: 27.17%
 Сохранен чекпоинт в: models/lstm_best.pt (val_loss=4.8279)

Some incorrect predictions:
Input: fuck i am going nuts due to this | True: exam | Predicted: weather
Input: nile ##yl ##over ##0 ##9 well im a little bit sick but im | True: fine | Predicted: sick
Input: just watched a romantic comedy made myself even sad ##der and now dan ##cin to the | True: music | Predicted: max
Input: crafts ##ter sweets ##4 ##ever false advertising those suitcase ##s are terribly cute but i was hoping for a mouse | True: too | Predicted: way
Input: sant ##ry ##rus ##h i can ##t believe im gonna miss his last show it might be better that way tho at least i won ##t be crying in the | True: aisles | Predicted: morning

Some correct predictions:
Input: ci ##ii ##nt your ##e | True: welcome | Predicted: welcome
Input: linda ##pants lo ##l you sims cheat ##er dude i never found bella in part | True: 2 | Predicted: 2
Input: fey ##bee me too 

## 4) Оценка DistilGPT2 (`eval_transformer_pipeline.py`) с ROUGE

In [1]:
import os, sys, importlib
sys.path.insert(0, os.path.join(os.getcwd(), "src"))
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import eval_transformer_pipeline as etp
importlib.reload(etp)
etp.main()  # печатает ROUGE и примеры


/root/code/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Eval GPT2 (nucleus, cap=48): 100%|██████████| 20000/20000 [09:32<00:00, 34.93it/s]


ROUGE: {'rouge1': 0.0664, 'rouge2': 0.0061, 'rougeL': 0.0657, 'rougeLsum': 0.0656}

Some CORRECT predictions:
CTX: Argh, why do I feel ill all
GEN:  the time?
REF:  the time?
------------------------------------------------------------
CTX: @MyOhSoSweetLife YOUR ON DR
GEN: UGS
REF: UGS
------------------------------------------------------------
CTX: YAY! new  friend
GEN: !
REF: !
------------------------------------------------------------
CTX: Unless it's single-thread
GEN: ed.
REF: ed.
------------------------------------------------------------
CTX: off to bed. and missing
GEN:  him
REF:  him
------------------------------------------------------------

Some INCORRECT predictions:
CTX: Ow, I burnt/froze my finger on
GEN:  the skin,
REF:  the toilet spray
------------------------------------------------------------
CTX: @allaboutanne We were shopping for food, haha.  How are you? 
GEN:    How do you know
REF:  Why you never call me?
--------------------------------------------------

# Выводы:
- LSTM (текущая версия) предсказывает один последний токен.

- DistilGPT-2 генерирует фрагмент текста (хвост).

Качество по примерам:
- LSTM: хорошо угадывает частые/служебные слова; смысловые концовки — нестабильно.

- DistilGPT-2 (ожидание): связнее и контекстнее, лучше удерживает тему хвоста.

Рекомендация:

- Для человекочитаемого автодополнения 1/4 текста — DistilGPT-2 (nucleus sampling top_p≈0.95, T≈0.7 или beam=4; max_new_tokens 48–64).

- Если критичны задержки/ресурсы и нужна «подсказка следующего токена» — LSTM.